# Aviso parroquial

Quiz sobre todo el tema de `Docker` para la siguiente clase (Lunes 17 de Octubre) antes de empezar la clase.

# Repaso de la clase pasada... Docker
1. Qué es `Docker`?
2. Qué es el `Dockerfile`?
3. Qué es una **imagen** de `Docker`?
4. Qué es un **contenedor** de `Docker`?
____

![](https://craft-code.com/wp-content/uploads/2021/08/docker_facebook_share.png)
## 1. Qué es Docker?
Es una herramienta para crear y desplegar ambientes aislados y correr aplicaciones con sus respectivas dependencias.

## 2. Qué es el `Dockerfile`?
Un archivo de texto que contiene una lista de comandos/instrucciones que se usan para crear la **imagen de Docker**.

## 3. Qué es una imagen de `Docker`?
Un plan para crear contenedores. Las imágenes son inmutables y todos los contenedores creados a partir de la misma imagen son exactamente iguales.

## 4. Qué es un **contenedor** de `Docker`?
Una instancia de la aplicación que esta ejecutándose.

### Analogía con galletas

![](https://miro.medium.com/max/1400/0*vA9VPPRkFtYvYdUY)

## Conceptos adicionales sobre `Docker`

### 1. Variantes de las imágenes
- **Standard**
```
FROM python:<version>
```

- **Slim**
Contiene lo mínimo necesario para correr `python`. Esta variante es adecuada cuando no requiere dependencias externas y trabaja bien con solo Python.
```
FROM python:<version>-slim
```

- **Alpine**
Basada en [Alpine Linux Project](https://alpinelinux.org/). Es adecuada cuando nos importa el tamaño de la imagen. La imagen base pesa 5MB.
```
FROM python:<version>-alpine
```
**Warning** Puede tener problemas con matplotlib debido a no usa la librería `glibc`.

- **windowsservercore**
Basada en Windows Server
```
FROM python:<version>-windowsservercore
```

### 2. Directorio de trabajo
Suponga que se tiene la siguiente estructura:

![](docker_directory.png)

Y que se tiene el siguiente `Dockerfile`

```
FROM python:3.7
WORKDIR /app
COPY ./requirements.txt requirements.txt
RUN pip install -r /requirements.txt
COPY ./app /app
CMD ["python", "main.py"]
```

También se puede tener este otro `Dockerfile`

```
FROM python:3.9
WORKDIR /code
COPY ./requirements.txt /code/requirements.txt
RUN pip install --no-cache-dir --upgrade -r /code/requirements.txt
COPY ./app /code/app
CMD ["uvicorn", "app.main:app", "--host", "0.0.0.0", "--port", "80"]
```

**WARNING:** Tener cuidado con la importación relativa de módulos (carpetas) personalizados.

### 3. Construcción incremental

Estas líneas
```
RUN python -m pip install -U pip
RUN pip install -r requirements.txt
```

Se pueden reemplazar por...
```
RUN python -m pip install -U pip && pip install -r requirements.txt
```



## API Key con FastAPI



In [1]:
from fastapi import Security, Depends, FastAPI, HTTPException
from fastapi.security.api_key import APIKeyQuery, APIKey

from starlette.status import HTTP_403_FORBIDDEN

In [2]:
API_KEY = "1234567asdfgh"
API_KEY_NAME = "access_token"

api_key_query = APIKeyQuery(name=API_KEY_NAME, auto_error=False)

def get_api_key(api_key_query: str = Security(api_key_query)):

    if api_key_query == API_KEY:
        return api_key_query
    else:
        raise HTTPException(
            status_code=HTTP_403_FORBIDDEN, detail="Could not validate credentials"
        )

@app.get("/secure_endpoint", tags=["test"])
async def get_open_api_endpoint(api_key: APIKey = Depends(get_api_key)):
    response = "How cool is this?"
    return response